In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer

import sklearn

from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.svm import SVC

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB, CategoricalNB

from IPython.display import clear_output

import pandas as pd
import numpy as np
import time
import string
from tqdm import notebook
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings("ignore")

# Helpers

In [2]:
def progress(done, left, start):
    # Print bar
    clear_output(wait=True)
    print(f"[{'#' * done}{' ' * left}] {round(done/(done+left) * 100, 2)}% ({done}/{done+left})")
    
    # Bereken gepasseerde tijd
    current = time.time()
    hours = int((current - start) // 3600)
    minutes = int((current - start) % 3600 // 60)
    seconds = int((current - start) % 60)
    
    # Print tijd
    print(f"""Time passed = {hours:02d}:{minutes:02d}:{seconds:02d}""")
    
    new_done = done + 1
    new_left = left - 1
    
    return new_done, new_left


# Laad data in

In [3]:
equal = pd.read_csv("../data/training_videos.csv").drop("Unnamed: 0", axis = 1)
equal["full_text"].fillna("", inplace=True)

equal.head()

,channel,video_id,conspiracy,full_text
0,UCH9gafn41oPg7s6bQwiMwTg,hgOdDCd6N10,False,islam beat femin right end stone begin motherf...
1,UCHr26UnUE_FHYtAXc_ddfgg,um-kBMGmW5k,False,24 hour burn man 2019 micro edit littl tast bu...
2,UCsu6NM_ARGWzDwL1C9xEhqA,9Z2CMOdLoB8,False,polic reform time turmoil injustic world cri o...
3,UCTjxmzChimJa3X_rAgLAnxg,GNJny3dtmoQ,False,career job career support video-mak donat matt...
4,UCaoSDiNkFmGQfvhvZMWxOHw,lOoCe0OZgvc,False,socrat abl predict civil unrest comment curren...


# Vectorize

In [4]:
# Vectorize words
v = TfidfVectorizer(max_df=.75, min_df=2)
x = v.fit_transform(equal['full_text'])

x

<44312x134844 sparse matrix of type '<class 'numpy.float64'>'
	with 22015145 stored elements in Compressed Sparse Row format>

In [5]:
x.shape

(44312, 134844)

In [6]:
X = x
Y = equal['conspiracy'].values.ravel()

# 10% of the data will be the test set
X_trainval, X_test, y_trainval, y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.1,\
                                                                                  random_state=0)

# of the remaining 90%, 90% will be training set
X_train, X_valid, y_train, y_valid = sklearn.model_selection.train_test_split(X_trainval, y_trainval,\
                                                                              train_size=0.9,\
                                                                              random_state=0)

# K-nearest neighbors

## Hyperparameter tuning

In [7]:
measures = {"K":[], "Accuracy":[], "Precision":[],"Recall":[], "F1":[]}

for k in notebook.tqdm(range(1, 16)):
    knn = KNeighborsClassifier(k).fit(X_train, y_train)

    y_pred = knn.predict(X_valid)

    measures["K"].append(k)
    measures["Accuracy"].append(sklearn.metrics.accuracy_score(y_valid, y_pred))
    measures["Precision"].append(sklearn.metrics.precision_score(y_valid, y_pred))
    measures["Recall"].append(sklearn.metrics.recall_score(y_valid, y_pred))
    measures["F1"].append(sklearn.metrics.f1_score(y_valid, y_pred))

df_knn = pd.DataFrame(measures)
df_knn.to_csv("k-nearest_neighbors.csv")

  0%|          | 0/15 [00:00<?, ?it/s]

## Optimal configuration

In [20]:
knn = KNeighborsClassifier(4).fit(X_train, y_train)

y_pred = knn.predict(X_valid)

print("Accuracy :", sklearn.metrics.accuracy_score(y_valid, y_pred))
print("Precision:", sklearn.metrics.precision_score(y_valid, y_pred))
print("Recall   :", sklearn.metrics.recall_score(y_valid, y_pred))
print("F1       :", sklearn.metrics.f1_score(y_valid, y_pred))

Accuracy : 0.8222166499498496
Precision: 0.8227848101265823
Recall   : 0.8304668304668305
F1       : 0.82660797260944


# Neural Network

## Hyperparameter tuning

In [9]:
measures = {"Activation function":[], "# of hidden layers":[], "Neurons per layer":[],
            "Accuracy":[], "Precision":[],"Recall":[], "F1":[]}

done = 0
left = 4*3*3
start = time.time()

for act in ['identity', 'logistic', 'tanh', 'relu']:
    for layers in [1, 10, 25]:
        for neurons in [1, 10, 20]:
            # Progress bar
            done, left = progress(done, left, start)
            
            # Fit
            mlp = MLPClassifier(hidden_layer_sizes=[neurons]*layers, activation=act,\
                    random_state=0).fit(X_train, y_train)

            # Predict en sla waardes op
            y_pred = mlp.predict(X_test)
            measures["Activation function"].append(act)
            measures["# of hidden layers"].append(layers)
            measures["Neurons per layer"].append(neurons)
            
            measures["Accuracy"].append(sklearn.metrics.accuracy_score(y_valid, mlp.predict(X_valid)))
            measures["Precision"].append(sklearn.metrics.precision_score(y_valid, mlp.predict(X_valid)))
            measures["Recall"].append(sklearn.metrics.recall_score(y_valid, mlp.predict(X_valid)))
            measures["F1"].append(sklearn.metrics.f1_score(y_valid, mlp.predict(X_valid)))
            
df_nn = pd.DataFrame(measures)
df_nn.to_csv("neural_network.csv")

[################################### ] 97.22% (35/36)
Time passed = 08:495:55


## Optimal configuration

In [9]:
mlp = MLPClassifier(hidden_layer_sizes=[10]*25, activation="identity",
                    random_state=0).fit(X_train, y_train)

y_pred = mlp.predict(X_valid)

print("Accuracy :", sklearn.metrics.accuracy_score(y_valid, y_pred))
print("Precision:", sklearn.metrics.precision_score(y_valid, y_pred))
print("Recall   :", sklearn.metrics.recall_score(y_valid, y_pred))
print("F1       :", sklearn.metrics.f1_score(y_valid, y_pred))

Accuracy : 0.8643430290872618
Precision: 0.8553758325404377
Recall   : 0.8835380835380835
F1       : 0.8692289098380469


# Support-vector machine

## Hyperparameter tuning

In [10]:
measures = {"Kernel":[], "C":[], "Accuracy":[], "Precision":[], "Recall":[], "F1":[]}

done = 0
left = 4*4
start = time.time()

for kernel in ["linear", "poly", "rbf", "sigmoid"]:
    for C in [0.1, 1, 10, 100]:       
        # Print progress
        done, left = progress(done, left, start)
        
        # Train
        svm = SVC(C=C, kernel=kernel, max_iter = 10000).fit(X_train, y_train)
        
        # Predict
        y_pred = svm.predict(X_valid)

        # Store
        measures["Kernel"].append(kernel)
        measures["C"].append(C)

        measures["Accuracy"].append(sklearn.metrics.accuracy_score(y_valid, y_pred))
        measures["Precision"].append(sklearn.metrics.precision_score(y_valid, y_pred))
        measures["Recall"].append(sklearn.metrics.recall_score(y_valid, y_pred))
        measures["F1"].append(sklearn.metrics.f1_score(y_valid, y_pred))
        
df_svm = pd.DataFrame(measures)
df_svm.to_csv("support-vector_machine.csv")

[############### ] 93.75% (15/16)
Time passed = 07:431:58


## Optimal configuration

In [22]:
svm = SVC(C=10, kernel="rbf", max_iter = 10000).fit(X_train, y_train)

y_pred = svm.predict(X_valid)

print("Accuracy :", sklearn.metrics.accuracy_score(y_valid, y_pred))
print("Precision:", sklearn.metrics.precision_score(y_valid, y_pred))
print("Recall   :", sklearn.metrics.recall_score(y_valid, y_pred))
print("F1       :", sklearn.metrics.f1_score(y_valid, y_pred))

Accuracy : 0.8884152457372116
Precision: 0.9068577277379734
Recall   : 0.8707616707616708
F1       : 0.8884432188518426


# Logistic Regression

## Hyperparameter tuning

In [7]:
measures = {"Penalty":[], "C":[], "Solver":[], "Accuracy":[], "Precision":[],"Recall":[], "F1":[]}

done = 0
left = 4*3*3
start = time.time()

for solver in ['newton-cg', 'lbfgs', 'sag', 'saga']:
    for pen in ['l1', 'l2', 'none']:
        for C in [1, 10, 20]:
            try:
                # Print progress
                done, left = progress(done, left, start)
                
                # Train solver
                logr = LogisticRegression(penalty=pen, 
                                          C=C, 
                                          solver=solver,
                                          n_jobs=-1).fit(X_train, y_train)

                # Predict
                y_pred = logr.predict(X_valid)

                measures["Solver"].append(solver)
                measures["Penalty"].append(pen)
                measures["C"].append(C)

                measures["Accuracy"].append(sklearn.metrics.accuracy_score(y_valid, y_pred))
                measures["Precision"].append(sklearn.metrics.precision_score(y_valid, y_pred))
                measures["Recall"].append(sklearn.metrics.recall_score(y_valid, y_pred))
                measures["F1"].append(sklearn.metrics.f1_score(y_valid, y_pred))
            except:
                pass
            
df_logr = pd.DataFrame(measures)
df_logr.to_csv("logistic_regression.csv")

[############### ] 93.75% (15/16)
Time passed = 00:04:44


## Optimal configuration

In [8]:
# Train solver
logr = LogisticRegression(penalty="l2", 
                          C=20, 
                          solver="saga",
                          n_jobs=-1).fit(X_train, y_train)

y_pred = logr.predict(X_valid)

print("Accuracy :", sklearn.metrics.accuracy_score(y_valid, y_pred))
print("Precision:", sklearn.metrics.precision_score(y_valid, y_pred))
print("Recall   :", sklearn.metrics.recall_score(y_valid, y_pred))
print("F1       :", sklearn.metrics.f1_score(y_valid, y_pred))

Accuracy : 0.8713640922768305
Precision: 0.8808808808808809
Recall   : 0.8648648648648649
F1       : 0.8727994049094967


# Ridge Classification

## Hyperparameter tuning

In [27]:
measures = {"Solver":[], "Alpha":[], "Accuracy":[], "Precision":[], "Recall":[], "F1":[]}

done = 0
left = 3*4
start = time.time()

for solver in ['auto', 'sparse_cg', 'sag']:
    for alpha in [0.1, 1, 10, 100]:
        # Print progress
        done, left = progress(done, left, start)
        
        ridge = RidgeClassifier(solver=solver, alpha=alpha).fit(X_train, y_train)

        y_pred = ridge.predict(X_valid)
        
        measures["Solver"].append(solver)
        measures["Alpha"].append(alpha)

        measures["Accuracy"].append(sklearn.metrics.accuracy_score(y_valid, y_pred))
        measures["Precision"].append(sklearn.metrics.precision_score(y_valid, y_pred))
        measures["Recall"].append(sklearn.metrics.recall_score(y_valid, y_pred))
        measures["F1"].append(sklearn.metrics.f1_score(y_valid, y_pred))

df_ridge = pd.DataFrame(measures)
df_ridge.to_csv("ridge_classification.csv")

[###########     ] 68.75% (11/16)
Time passed = 00:01:05


## Optimal configuration

In [13]:
ridge = RidgeClassifier(solver="auto", alpha=1).fit(X_train, y_train)

y_pred = ridge.predict(X_valid)

print("Accuracy :", sklearn.metrics.accuracy_score(y_valid, y_pred))
print("Precision:", sklearn.metrics.precision_score(y_valid, y_pred))
print("Recall   :", sklearn.metrics.recall_score(y_valid, y_pred))
print("F1       :", sklearn.metrics.f1_score(y_valid, y_pred))

Accuracy : 0.8688565697091274
Precision: 0.8841463414634146
Recall   : 0.855036855036855
F1       : 0.8693479890082438


# Hybrid Machine Learning

In [23]:
# Predict the label for every video with every optimal model
hybrid = equal[["video_id", "conspiracy"]]
hybrid["ridge"] = ridge.predict(x)
hybrid["svm"] = svm.predict(x)
hybrid["neural_network"] = mlp.predict(x)
hybrid["log_reg"] = logr.predict(x)
hybrid["knn"] = knn.predict(x)

hybrid.head()

,video_id,conspiracy,ridge,svm,neural_network,log_reg,knn
0,hgOdDCd6N10,False,False,False,False,False,False
1,um-kBMGmW5k,False,False,False,False,False,False
2,9Z2CMOdLoB8,False,False,False,False,False,False
3,GNJny3dtmoQ,False,False,False,False,False,False
4,lOoCe0OZgvc,False,False,False,False,False,False


In [40]:
# For every video, find the majority vote
models = ["ridge", "svm", "neural_network", "log_reg", "knn"]
hybrid["democratic"] = hybrid[models].sum(axis=1) >= 3

In [43]:
# get predictions and true values
y = hybrid["conspiracy"].values.ravel()
dem_pred = hybrid["democratic"].values

In [47]:
print("accuracy :", sklearn.metrics.accuracy_score(dem_pred, Y2))
print("precision:", sklearn.metrics.precision_score(dem_pred, Y2))
print("recall   :", sklearn.metrics.recall_score(dem_pred, Y2))
print("f1       :", sklearn.metrics.f1_score(dem_pred, Y2))

accuracy : 0.9693988084491786
precision: 0.9666004693988085
recall   : 0.9720406681190995
f1       : 0.969312935638635
